# PROJECT TITLE


#### PROJECT DESCRIPTION


The following is the relationship entity diagram for the WORLD database which has three tables:

- city

- country

- countrylanguage

- northamerica


In [2]:
import pandas as pd 
import numpy as np 
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

In [3]:
# create a connection 
conn = mysql.connector.connect(host='localhost', port='3306',user='root',password='****',database='world')

In [4]:
# show all the tables in the database 

pd.read_sql_query('SHOW TABLES', conn)

,Tables_in_world
0,city
1,country
2,countrylanguage
3,northamerica


In [8]:
# checking the first few rows of the city table
city_df = pd.read_sql_query('SELECT * FROM city', conn)

city_df.head()

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


In [9]:
# checking the first few rows of the country table
country_df = pd.read_sql_query('SELECT * FROM country', conn)

country_df.head()

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL


In [10]:
countrylanguage_df = pd.read_sql_query('SELECT * FROM countrylanguage', conn)

countrylanguage_df.head()

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9


In [28]:
#Average Population of Cities in Each Region
avg_pop_region = pd.read_sql_query('SELECT c.region, AVG(ci.population) AS avg_population FROM country c JOIN city ci ON c.code = ci.countrycode GROUP BY c.region ORDER BY avg_population DESC;',conn)

In [29]:
avg_pop_region

,region,avg_population
0,Central Africa,501625.6579
1,Australia and New Zealand,487534.6667
2,Eastern Asia,429022.3432
3,Northern Africa,425975.6275
4,Middle East,404433.1839
5,Southern and Central Asia,374214.3604
6,South America,366037.9979
7,Southeast Asia,343660.6902
8,Central America,336025.3265
9,Eastern Africa,334264.8056


In [31]:
# Countries where english is spoken as the official language 

english_speaking = pd.read_sql_query("SELECT c.name FROM country c JOIN countrylanguage cl ON c.code = cl.countrycode WHERE cl.language = 'English' AND cl.IsOfficial = 'T';",conn)

In [33]:
english_speaking.head()

,name
0,Anguilla
1,American Samoa
2,Antigua and Barbuda
3,Australia
4,Belize


In [34]:
# Cities with Population Higher than average of cities population in the same Country 

cities_greater_average = pd.read_sql_query("SELECT ci.name AS city_name, ci.population, c.name AS country_name, c.population AS country_population FROM city ci JOIN country c ON ci.countrycode = c.code WHERE ci.population > (SELECT AVG(population) FROM city WHERE countrycode = ci.countrycode);",conn)

In [36]:
cities_greater_average.head()

,city_name,population,country_name,country_population
0,Kabul,1780000,Afghanistan,22720000
1,Luanda,2022000,Angola,12878000
2,South Hill,961,Anguilla,8000
3,Dubai,669181,United Arab Emirates,2441000
4,Abu Dhabi,398695,United Arab Emirates,2441000


In [41]:
# percentage of world population in each continent 

percentage_continent = pd.read_sql_query("SELECT c.continent, (SUM(ci.population) / (SELECT SUM(population) FROM city)) * 100 AS percentage_of_world_population FROM country c JOIN city ci ON c.code = ci.countrycode GROUP BY c.continent;",conn)

In [42]:
percentage_continent

,continent,percentage_of_world_population
0,North America,11.7694
1,Asia,48.7985
2,Africa,9.5021
3,Europe,16.9243
4,South America,12.0343
5,Oceania,0.9714


In [44]:
# top 5 languages spoken worldwide by percentage

top_five_languages = pd.read_sql_query("SELECT language, SUM(percentage) AS total_percentage FROM countrylanguage GROUP BY language ORDER BY total_percentage DESC LIMIT 5;",conn)

In [45]:
top_five_languages

,language,total_percentage
0,Spanish,1849.4
1,Arabic,1578.9
2,English,1126.5
3,Creole English,794.8
4,Creole French,540.3


Returning results for a View to Show Countries and Their Official Languages

`CREATE VIEW CountryOfficialLanguages AS

SELECT c.name, cl.language

FROM country c

JOIN countrylanguage cl ON c.code = cl.countrycode

WHERE cl.IsOfficial = 'T';
`


In [47]:
# Returning results for a View to Show Countries and Their Official Languages

official_languages = pd.read_sql_query("SELECT * FROM CountryOfficialLanguages;",conn)

In [49]:
official_languages.head(10)

,name,language
0,Aruba,Dutch
1,Afghanistan,Dari
2,Afghanistan,Pashto
3,Anguilla,English
4,Albania,Albaniana
5,Andorra,Catalan
6,Netherlands Antilles,Dutch
7,Netherlands Antilles,Papiamento
8,United Arab Emirates,Arabic
9,Argentina,Spanish


Stored Procedure to Get Country Details by Continent

`DELIMITER //

CREATE PROCEDURE GetCountryDetailsByContinent(IN continentName VARCHAR(255))

BEGIN

    SELECT * FROM country WHERE continent = continentName;

END //

DELIMITER ;
`


In [ ]:
# using stored procedure to get country details from countries in Europe 

Europe =pd.read_sql_query("CALL GetCountryDetailsByContinent('Europe');",conn)

In [52]:
Europe.head(10)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34,AL
1,AND,Andorra,Europe,Southern Europe,468.0,1278.0,78000,83.5,1630.0,NaN,Andorra,Parliamentary Coprincipality,,55,AD
2,AUT,Austria,Europe,Western Europe,83859.0,1918.0,8091800,77.7,211860.0,206025.0,Österreich,Federal Republic,Thomas Klestil,1523,AT
3,BEL,Belgium,Europe,Western Europe,30518.0,1830.0,10239000,77.8,249704.0,243948.0,België/Belgique,"Constitutional Monarchy, Federation",Albert II,179,BE
4,BGR,Bulgaria,Europe,Eastern Europe,110994.0,1908.0,8190900,70.9,12178.0,10169.0,Balgarija,Republic,Petar Stojanov,539,BG
5,BIH,Bosnia and Herzegovina,Europe,Southern Europe,51197.0,1992.0,3972000,71.5,2841.0,NaN,Bosna i Hercegovina,Federal Republic,Ante Jelavic,201,BA
6,BLR,Belarus,Europe,Eastern Europe,207600.0,1991.0,10236000,68.0,13714.0,NaN,Belarus,Republic,Aljaksandr Lukašenka,3520,BY
7,CHE,Switzerland,Europe,Western Europe,41284.0,1499.0,7160400,79.6,264478.0,256092.0,Schweiz/Suisse/Svizzera/Svizra,Federation,Adolf Ogi,3248,CH
8,CZE,Czech Republic,Europe,Eastern Europe,78866.0,1993.0,10278100,74.5,55017.0,52037.0,¸esko,Republic,Václav Havel,3339,CZ
9,DEU,Germany,Europe,Western Europe,357022.0,1955.0,82164700,77.4,2133367.0,2102826.0,Deutschland,Federal Republic,Johannes Rau,3068,DE
